<img width = 85% src='rapids_motivation.png'>

<img width = 75% src='choices.png'>

### Overview 
1 -- synthetically generate a learning problem [ classification ]

2 -- go through a data science pipeline [ pre-processing , splitting , viz ]

3 -- model building [ xgboost ]
> hyper-parameters [ max-depth, nTrees, learning rate, regularization ... ]

> demonstrate performance [ CPU vs 1 GPU ]

4 -- scaling and hyper-parameter search
> dask + rapids [ xgboost ]
    
5 -- visualize search and reveal best model parameters

TODO - generate figures that caputre benefit of GPU scaling

6 -- extensions[ multi-node [ dask kubernetes ], dask_xgboost [ larger dataset ] ]

In [ ]:
import os
import sys
from enum import Enum

if sys.version_info[0] >= 3:
    from urllib.request import urlretrieve  # pylint: disable=import-error,no-name-in-module
else:
    from urllib import urlretrieve  # pylint: disable=import-error,no-name-in-module

class LearningTask(Enum):
    REGRESSION = 1
    CLASSIFICATION = 2
    MULTICLASS_CLASSIFICATION = 3

class Data:  # pylint: disable=too-few-public-methods,too-many-arguments
    def __init__(self, X_train, X_test, y_train, y_test, learning_task, qid_train=None,
                 qid_test=None):
        self.X_train = X_train
        self.X_test = X_test
        self.y_train = y_train
        self.y_test = y_test
        self.learning_task = learning_task
        # For ranking task
        self.qid_train = qid_train
        self.qid_test = qid_test
 
      
        

In [ ]:
import numpy as np; import numpy.matlib
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np; import numpy.matlib

In [ ]:
import ipyvolume as ipv

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# algos, datasets
import xgboost; from xgboost import plot_tree
from sklearn import datasets; from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
import cupy
import cudf

In [ ]:
import time

In [ ]:
import rapids_lib_v10 as rl
''' NOTE: anytime changes are made to rapids_lib.py you can either:
      1. refresh/reload via the code below, OR
      2. restart the kernel '''
import importlib; importlib.reload(rl)

# Data retrieving


In [ ]:
def prepare_higgs(dataset_folder, nrows):
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz'
    local_url = os.path.join(dataset_folder, os.path.basename(url))
    pickle_url = os.path.join(dataset_folder,
                              "higgs" + ("" if nrows is None else "-" + str(nrows)) + ".pkl")

    if os.path.exists(pickle_url):
        return pickle.load(open(pickle_url, "rb"))

    if not os.path.isfile(local_url):
        urlretrieve(url, local_url)
    higgs = pd.read_csv(local_url, nrows=nrows, header = None, 
                        names= ['label','lepton_pT','lepton_eta','lepton_phi','missing_energy_magnitude','missing_energy_phi','jet_1_pt',
                             'jet_1_eta','jet_1_phi','jet_1_b_tag','jet_2_pt','jet_2_eta','jet_2_phi','jet_2_b_tag',
                             'jet_3_pt','jet_3_eta','jet_3_phi','jet_3_b-tag','jet_4_pt','jet_4_eta','jet_4_phi',
                             'jet_4_b_tag','m_jj','m_jjj','m_lv','m_jlv','m_bb','m_wbb','m_wwbb'] )
    X = higgs.iloc[:, 1:]
    y = higgs.iloc[:, 0]
    

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=77,
                                                        test_size=0.2,
                                                        )
    data = Data(X_train, X_test, y_train, y_test, LearningTask.CLASSIFICATION)
    pickle.dump(data, open(pickle_url, "wb"), protocol=4)
    return data

In [ ]:
num_rows = 11000000
higgs_df = prepare_higgs(".", num_rows)

In [ ]:
print('Number of rows in training dataset:', len(higgs_df.X_train))
print('Number of rows in testing dataset:',len(higgs_df.X_test))

# Helix and Whirl Datasets

In [ ]:
trainData_pDF = higgs_df.X_train
testData_pDF = higgs_df.X_test
trainLabels_pDF = higgs_df.y_train.to_frame()
testLabels_pDF = higgs_df.y_test.to_frame()

trainData_cDF = cudf.DataFrame.from_pandas(trainData_pDF)
testData_cDF = cudf.DataFrame.from_pandas(testData_pDF)
trainLabels_cDF = cudf.DataFrame.from_pandas(trainLabels_pDF)
testLabels_cDF = cudf.DataFrame.from_pandas(testLabels_pDF)

# Split train and test data 

In [ ]:
expLog = {}

# Training and Evaluating a Single Model on CPU and GPU

In [ ]:
''' -------------------------------------------------------------------------
>  GPU Train and Test
------------------------------------------------------------------------- '''
def train_model_GPU (trainData_cDF, testData_cDF, paramsGPU = {}):    
    print('training xgboost model on GPU');  
    startTime = time.time()    
    
    trainDMatrix = xgboost.DMatrix( trainData_cDF.to_pandas(), label = trainLabels_cDF.to_pandas())    
    trainedModelGPU = xgboost.train( dtrain = trainDMatrix, params = paramsGPU, num_boost_round = paramsGPU['num_boost_rounds'] )
    
    return trainedModelGPU, time.time() - startTime

def test_model_GPU ( trainedModelGPU, testData_cDF, testLabels_cDF ):
    print('testing xgboost model on GPU')
    startTime = time.time()   
    
    testDMatrix = xgboost.DMatrix( data = testData_cDF.to_pandas(), label = testLabels_cDF.to_pandas())    
    predictionsGPU = trainedModelGPU.predict(testDMatrix)
    
    return predictionsGPU, time.time() - startTime

''' -------------------------------------------------------------------------
>  CPU Train and Test
------------------------------------------------------------------------- '''
def train_model_CPU (trainData_cDF, trainLabels_cDF, paramsCPU = {}):    
    print('training xgboost model on {} CPU cores'.format(nCores) )

    startTime = time.time()
    
    trainDMatrix = xgboost.DMatrix( trainData_cDF.to_pandas(), label = trainLabels_cDF.to_pandas())
    
    xgBoostModelCPU = xgboost.train( dtrain = trainDMatrix, params = paramsCPU, num_boost_round = paramsCPU['num_boost_rounds'])
    
    return xgBoostModelCPU, time.time() - startTime

def test_model_CPU ( trainedModelCPU, testData_cDF, testLabels_cDF ):
    print('testing xgboost model on CPU')
    startTime = time.time()
    
    testDMatrix = xgboost.DMatrix( testData_cDF.to_pandas(), label = testLabels_cDF.to_pandas())
    predictionsCPU = trainedModelCPU.predict(testDMatrix)
    
    return predictionsCPU, time.time() - startTime

### Define model parameters
https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
nCores = !nproc --all
nCores = int(nCores[0])

paramsCPU = {
    'max_depth': 3,
    'num_boost_rounds': 100,    
    'learning_rate': .1,
    'lambda': 1,    
    'objective': 'binary:hinge',
    'tree_method': 'hist',
    'n_jobs': nCores,
    'random_state': 0
}

paramsGPU = {
    'max_depth': 3,
    'num_boost_rounds': 100,
    'learning_rate': .1,
    'lambda': 1,    
    'objective': 'binary:hinge',
    'tree_method': 'gpu_hist',
    'n_gpus': 1,    
    'random_state': 0
}

### **CPU** Training and Inference

In [ ]:
trainedModelCPU, t_trainCPU = train_model_CPU ( trainData_cDF, trainLabels_cDF, paramsCPU )

In [ ]:
predictionsCPU, t_inferCPU = test_model_CPU ( trainedModelCPU, testData_cDF, testLabels_cDF)

In [ ]:
expLog = rl.update_log( expLog, [['CPU_model_training', t_trainCPU], ['CPU_model_inference', t_inferCPU]] ); 

### **GPU** Model Training and Inference

In [ ]:
trainedModelGPU, t_trainGPU = train_model_GPU ( trainData_cDF, trainLabels_cDF, paramsGPU )

In [ ]:
predictionsGPU, t_inferGPU = test_model_GPU ( trainedModelGPU, testData_cDF, testLabels_cDF)

In [ ]:
expLog = rl.update_log( expLog, [['GPU_model_training', t_trainGPU], ['GPU_model_inference', t_inferGPU]] ); 

### Compare CPU and GPU Accuracy [ plot on train and test ]

In [ ]:
trainLabels_pDF = trainLabels_cDF.to_pandas()
testLabels_pDF = testLabels_cDF.to_pandas()

trainPredictionsCPU, _ = test_model_CPU ( trainedModelCPU, trainData_cDF, trainLabels_cDF)
trainPredictionsGPU, _ = test_model_GPU ( trainedModelGPU, trainData_cDF, trainLabels_cDF)
trainAccuracyCPU = accuracy_score(trainLabels_pDF, trainPredictionsCPU)
trainAccuracyGPU = accuracy_score(trainLabels_pDF, trainPredictionsGPU)
print( 'train data CPU accuracy = {}'.format( trainAccuracyCPU ) )
print( 'train data GPU accuracy = {}'.format( trainAccuracyGPU ) )

accuracyCPU = accuracy_score(testLabels_pDF, predictionsCPU)
accuracyGPU = accuracy_score(testLabels_pDF, predictionsGPU)
print( 'test data CPU accuracy = {}'.format( accuracyCPU ) )
print( 'test data GPU accuracy = {}'.format( accuracyGPU ) )

In [ ]:
cMat = confusion_matrix(testLabels_pDF, predictionsGPU)
print(cMat)

### Visualize the first decision tree

In [ ]:
%%time
fig = plt.figure(figsize=(100,50))
plot_tree(trainedModelGPU, num_trees=0, ax=plt.subplot(1,1,1))

In [ ]:
expLog

# Parameter Search          
> TODO: ADD early stopping 

### Initialize Dask

In [ ]:
kube_ip = !echo $KUBEFLOW_CONTROLLER_SERVICE_HOST
if kube_ip[0]:
    dask_type = "k8s"
else:
    dask_type = "local"

In [ ]:
# !pip install dask_kubernetes

In [ ]:
import dask
from dask import delayed
from dask_cuda import LocalCUDACluster
from dask_kubernetes import KubeCluster
from dask.distributed import Client
from time import sleep # We need to give K8S some time to manually scale workers

In [ ]:
import xgboost;
print( xgboost.__version__ )

#### Define the Kubernetes Dask Spec

In [ ]:
worker_spec_fname = '/worker_spec.yaml'
worker_spec = '''
# worker-spec.yml

kind: Pod
metadata:
  labels:
    foo: bar
spec:
  restartPolicy: Never
  containers:
  - image: supertetelman/k8s-rapids-dask:0.9-cuda10.0-runtime-ubuntu18.04
    imagePullPolicy: IfNotPresent
    args: [dask-worker,  --nthreads, '1', --no-bokeh, --memory-limit, 6GB, --death-timeout, '60']
    name: dask
    resources:
      limits:
        cpu: "2"
        memory: 6G
        nvidia.com/gpu: 1
      requests:
        cpu: "2"
        memory: 6G
        nvidia.com/gpu: 1
'''

with open(worker_spec_fname, "w") as yaml_file:
    yaml_file.write(worker_spec)

In [ ]:
def scale_cluster(cluster, n, m=None):
    if dask_type != "k8s":
        print("Cannot scale non-K8S cluster")
        return
    if m is None:
        print(f'Scaling to {n} nodes')
        cluster.scale(n)
    else:
        print(f'Setting Scaling to adapatve, [{n}, {m}]')
        cluster.adapt(minimum = n, maximum = m)
    sleep(10)

### Set number of workers [ changes require kernel restart ]

In [ ]:
if dask_type == "k8s":
    cluster = KubeCluster.from_yaml(worker_spec_fname)
    scale_cluster(cluster, 4)
else:
    cluster = LocalCUDACluster(ip="", n_workers = 4)

client = Client(cluster)

In [ ]:
client

In [ ]:
cluster

### Pre-scatter Demo

In [ ]:
if client is not None:        
    scatteredData_future = client.scatter( [ trainData_cDF, testData_cDF ], broadcast = True)

In [ ]:
scatteredData_future

In [ ]:
print ( '{} \n {} \n'.format( scatteredData_future[0].key, scatteredData_future[1].key ) )
client.who_has(scatteredData_future)

## HPO Strategy - Particle Swarm [ Explore + Exploit ]

In [ ]:
def update_particles( paramRanges, particlesInTimestep, velocitiesInTimestep, bestParamsIndex, globalBestParams, sBest = .85, sExplore = .1 , deltaTime = 1, randomSeed = None):
    
    nParticles = particlesInTimestep.shape[ 0 ]
    nParameters = particlesInTimestep.shape[ 1 ]    
        
    globalBestRepeated = numpy.matlib.repmat( np.array( globalBestParams ).reshape( -1, 1 ), nParticles, 1).reshape( nParticles, nParameters )    
    
    if randomSeed is not None:
        np.random.seed(randomSeed)
        
    # move to best + explore | globalBest + personalBest
    velocitiesInTimestep += 0. * velocitiesInTimestep + sBest * ( globalBestRepeated - particlesInTimestep ) \
                            + sExplore * ( np.random.randn( nParticles, nParameters ) )
    
    particlesInTimestep += velocitiesInTimestep * deltaTime 
    
    # TODO: avoid duplicates
    
    # enforce param bounds
    for iParam in range( nParameters ):
        particlesInTimestep[ :, iParam ] = np.clip(particlesInTimestep[ :, iParam ], paramRanges[iParam][1], paramRanges[iParam][2])
        if paramRanges[iParam][3] == 'int':
            particlesInTimestep[ :, iParam ] = np.round( particlesInTimestep[ :, iParam ] )
            
    return particlesInTimestep, velocitiesInTimestep
    

## HPO Harness
> Particle Evals [ Train and Test Logic ]

In [ ]:
def train_model_hpo ( trainData_cDF, trainLabels_cDF, testData_cDF, testLabels_cDF, particleParams, iParticle, iTimestep ):
    
    # fixed parameters
    paramsGPU = { 'objective': 'binary:hinge',
                  'tree_method': 'gpu_hist',
                  'n_gpus': 1,
                  'random_state': 0 }
    
    # parameters to search over
    paramsGPU['max_depth'] = int(particleParams[0])
    paramsGPU['learning_rate'] = particleParams[1]
    paramsGPU['lambda'] = particleParams[2]
    paramsGPU['num_boost_rounds'] = 1000
    
    startTime = time.time()
    trainDMatrix = xgboost.DMatrix( data = trainData_cDF, label = trainLabels_cDF )
    testDMatrix = xgboost.DMatrix( data = testData_cDF, label = testLabels_cDF )
    trainedModelGPU = xgboost.train( dtrain = trainDMatrix, evals = [(testDMatrix, 'test')], 
                                     params = paramsGPU,
                                     num_boost_round = paramsGPU['num_boost_rounds'],
                                     early_stopping_rounds = 15,
                                     verbose_eval = False )
    
    elapsedTime = time.time() - startTime
    
    return trainedModelGPU, elapsedTime

def test_model_hpo ( trainedModelGPU, trainingTime, testData_cDF, testLabels_cDF ):
    
    startTime = time.time()
    
    testDMatrix = xgboost.DMatrix( data = testData_cDF, label = testLabels_cDF )    
    predictionsGPU = trainedModelGPU.predict( testDMatrix ).astype(int)
    
    return predictionsGPU, trainedModelGPU.best_iteration, trainingTime, time.time() - startTime

In [ ]:
def run_hpo ( daskClient, nTimesteps, nParticles, paramRanges, trainData_cDF, trainLabels_cDF, testData_cDF, testLabels_cDF, randomSeed = 0, plotFlag = True):
    
    pandasTestLabels = testLabels_cDF.to_pandas()

    if daskClient is not None:
        scatteredData_future = daskClient.scatter( [ trainData_cDF, trainLabels_cDF, testData_cDF, testLabels_cDF ], broadcast = True )
    
    trainData_cDF_future = scatteredData_future[0]; trainLabels_cDF_future = scatteredData_future[1]
    testData_cDF_future = scatteredData_future[2]; testLabels_cDF_future = scatteredData_future[3]
    
    particles, velocities, accuracies, bestParticleIndex, \
        globalBestParticleParams, particleBoostingRounds, particleColors = rl.initialize_particle_swarm ( nTimesteps = nTimesteps, 
                                                                                                          nParticles = nParticles,
                                                                                                          paramRanges = paramRanges,
                                                                                                          randomSeed = randomSeed, 
                                                                                                          plotFlag = plotFlag)
    globalBestAccuracy = 0
    
    trainingTimes = np.zeros (( nTimesteps, nParticles ))    
    startTime = time.time()
    
    predictionHistory = np.zeros((nTimesteps, nParticles, testData_cDF.shape[0]))
    
    for iTimestep in range (0, nTimesteps ):    
        if daskClient is not None:
            # [ delayed ] train xgboost models on train data
            delayedParticleTrain = [ delayed( train_model_hpo )( trainData_cDF_future, trainLabels_cDF_future, 
                                                                     testData_cDF_future, testLabels_cDF_future, 
                                                                     particles[iTimestep, iParticle, : ], 
                                                                         iParticle, iTimestep) for iParticle in range(nParticles) ]

            # [ delayed ] determine number of trees/training-rounds returned early stopping -- used to set particle sizes
            delayedParticleRounds = [ iParticle[0].best_iteration for iParticle in delayedParticleTrain ]
            
            # [delayed ] eval trained models on test/validation data
            delayedParticlePredictions = [ delayed( test_model_hpo )(iParticle[0], iParticle[1], 
                                                                     testData_cDF_future, 
                                                                     testLabels_cDF_future) for iParticle in delayedParticleTrain ]
            
            # execute delayed             
            particlePredictions = dask.compute( delayedParticlePredictions )[0]            
            
            
            for iParticle in range(nParticles):
                predictionHistory[iTimestep, iParticle, :] = particlePredictions[iParticle][0]
            #import pdb; pdb.set_trace()
            
            # compute accuracies of predictions
            accuracies[iTimestep, :] = [ accuracy_score ( pandasTestLabels, iParticle[0]) for iParticle in particlePredictions ]
            particleBoostingRounds[iTimestep, : ] = [ iParticle[1] for iParticle in particlePredictions ]
            trainingTimes[iTimestep, :] = [ iParticle[2] for iParticle in particlePredictions ]
            del particlePredictions
        else:
            for iParticle in range(nParticles):
                trainedModels, _ = train_model_hpo ( pandasTrainData, pandasTrainLabels, particles[iTimestep, iParticle, : ], iParticle, iTimestep)
                predictions, _ = test_model_hpo( trainedModels, pandasTestData, pandasTestLabels)            
                accuracies[iTimestep, iParticle] = accuracy_score (pandasTestLabels, predictions)
        
        bestParticleIndex[iTimestep+1] = np.argmax( accuracies[iTimestep, :] )
        currentBestAccuracy = np.max( accuracies[iTimestep, :] )

        print('@ hpo timestep : {}, best accuracy is {}'.format(iTimestep, np.max(accuracies[iTimestep, :])) )
        if iTimestep +1 < nTimesteps:            
            if currentBestAccuracy > globalBestAccuracy:
                print('\t updating best GLOBAL accuracy')
                globalBestAccuracy = currentBestAccuracy
                globalBestParticleParams[iTimestep+1] = particles[iTimestep, bestParticleIndex[iTimestep+1], :]
            else:
                globalBestParticleParams[iTimestep+1] = globalBestParticleParams[iTimestep].copy()
            
            particles[iTimestep+1, :, :], velocities[iTimestep+1, :, : ] = update_particles ( paramRanges, 
                                                                                              particles[iTimestep, :, :].copy(),
                                                                                              velocities[iTimestep, :, :].copy(), 
                                                                                              bestParticleIndex[iTimestep+1], 
                                                                                              globalBestParticleParams[iTimestep+1], randomSeed = iTimestep)
            

    
    particleSizes = particleBoostingRounds/np.max(particleBoostingRounds)*10 + 2
    
    elapsedTime = time.time() - startTime
    
    bestParamIndex = np.unravel_index(np.argmax(accuracies, axis=None), accuracies.shape)

    # best accuracy
    print('highest accuracy               :  {} '.format(accuracies[bestParamIndex[0], bestParamIndex[1]]))
    print('   @ timestep {}, particle {} '.format( bestParamIndex[0], bestParamIndex[1]))

    print('\nbest model tree depth          :  {} '.format(particles[bestParamIndex[0], bestParamIndex[1], 0]))
    print('best model learning rate       :  {} '.format(particles[bestParamIndex[0], bestParamIndex[1], 1]))
    print('best model regularization      :  {} '.format(particles[bestParamIndex[0], bestParamIndex[1], 2]))
    print('best model num boosting rounds :  {} '.format(int(particleBoostingRounds[bestParamIndex[0], bestParamIndex[1]])))
    
    print( 'elapsed time : {}'.format(elapsedTime) )
    
    return accuracies, particles, velocities, particleSizes, particleColors, bestParticleIndex, bestParamIndex, particleBoostingRounds, trainingTimes, predictionHistory, elapsedTime

## Settings for HPO

In [ ]:
nTimesteps = 1
nParticles = 32

In [ ]:
paramRanges = { 0: ['max_depth', 3, 15, 'int'],
                1: ['learning_rate', .001, 1, 'float'],
                2: ['lambda', 0, 10, 'float'] }

### Run HPO

In [ ]:
# Kubernetes let's us dynamically scale the cluster size, which allows us to do these benchmarks
if dask_type == "k8s":
    scale_cluster(cluster, 1)
    run_hpo ( client, nTimesteps, nParticles, paramRanges, trainData_cDF,
              trainLabels_cDF, testData_cDF, testLabels_cDF, plotFlag=False )
    
    scale_cluster(cluster, 2)
    run_hpo ( client, nTimesteps, nParticles, paramRanges, trainData_cDF,
              trainLabels_cDF, testData_cDF, testLabels_cDF, plotFlag=False )
    
    scale_cluster(cluster, 0, 4)
    run_hpo ( client, nTimesteps, nParticles, paramRanges, trainData_cDF,
              trainLabels_cDF, testData_cDF, testLabels_cDF, plotFlag=False )
    
    scale_cluster(cluster, 0)

In [ ]:
scale_cluster(cluster, 4)

In [ ]:
accuracies, particles, velocities, particleSizes, particleColors, bestParticleIndex, \
bestParamIndex, particleBoostingRounds, trainingTimes, predictionHistory, elapsedTime = run_hpo ( client, nTimesteps, 
                                                                                                  nParticles,
                                                                                                  paramRanges, 
                                                                                                  trainData_cDF, trainLabels_cDF, testData_cDF, testLabels_cDF )

### Visualize search

In [ ]:
rl.viz_search( accuracies, particleBoostingRounds )

In [ ]:
rl.hpo_animate (particles, particleSizes, particleColors, paramRanges, nTimesteps = nTimesteps )

# Visualize the Prediction Trajectory of the Top Particle [ ending at its best timestep ]

In [ ]:
rl.plot_particle_learning ( nTimesteps, nParticles, testData_pDF, bestParamIndex, predictionHistory )

## Appendix
##### Jupyter Extensions
* [Dask Extension](https://github.com/dask/dask-labextension)
* [NVIDIA System Dashboard](https://github.com/jacobtomlinson/jupyterlab-nvdashboard/blob/master/README.md)

##### Dask Kubernetes
* [Example Notebook](https://github.com/supertetelman/k8s-rapids-dask/blob/master/k8s_examples/Scaling%20Dask%20in%20Kubernetes.ipynb)
* [Docker Image](https://github.com/supertetelman/k8s-rapids-dask/blob/master/Dockerfile)
* [K8S/Kubeflow/Dask Deployment Guide](https://github.com/NVIDIA/deepops/blob/master/docs/kubernetes-cluster.md)
* [Official Docs](https://kubernetes.dask.org/en/latest/)

In [ ]:
# 1 -- use non-synthetic dataset
# 2 -- kubernetes